In [1]:
# https://docs.min.io/docs/python-client-api-reference.html
    
from minio import Minio
client = Minio("bi.s3.es.gov.br", "BI@Prodest", "!BI@Prodest2021!")

In [2]:
buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name)

covid19
covid19-isolamento
imunizacao
incaper-agropec
indicadores-criminais
investimentos
observatorio-mulher
prodest-financeiro
sectides-investimentos
sedes-planoes-bandes
sedes-planoes-banestes
sedes-planoes-sep
sedh
sedu-covid
sedu-progefe
sedu-sigefes
sep
sesa-geomorb


In [3]:
objects = client.list_objects("sedh")
for obj in objects:
    print(obj.object_name)



MICRODADOS_BAIRROS.csv
OCID_INDICADORES_atendimentos_e_pessoas.csv


In [4]:
try:
    response = client.get_object("sedh", "OCID_INDICADORES_atendimentos_e_pessoas.csv")
finally:
    response.close()
    response.release_conn()

with client.select_object_content(
        "sedh",
        "OCID_INDICADORES_atendimentos_e_pessoas.csv",
        SelectRequest(
            "select * from S3Object",
            CSVInputSerialization(),
            CSVOutputSerialization(),
            request_progress=True,
        ),
) as result:
    for data in result.stream():
        print(data.decode())
    print(result.stats())

NameError: name 'SelectRequest' is not defined

In [ ]:
resp = client.select_object_content(
    Bucket='sedh',
    Key='OCID_INDICADORES_atendimentos_e_pessoas.csv',
    ExpressionType='SQL',
    Expression="SELECT * FROM s3object s ",
    InputSerialization = {'CSV': {"FileHeaderInfo": "Use"}},
    OutputSerialization = {'CSV': {}},
)

for event in resp['Payload']:
    if 'Records' in event:
        records = event['Records']['Payload'].decode('utf-8')
        print(records)
    elif 'Stats' in event:
        statsDetails = event['Stats']['Details']
        print("Stats details bytesScanned: ")
        print(statsDetails['BytesScanned'])
        print("Stats details bytesProcessed: ")
        print(statsDetails['BytesProcessed'])
        print("Stats details bytesReturned: ")
        print(statsDetails['BytesReturned'])

In [4]:
from minio import Minio
import pandas as pd
from csv import DictReader
import io

client = Minio("bi.s3.es.gov.br", "BI@Prodest", "!BI@Prodest2021!")

response = client.get_object("sedh", "OCID_INDICADORES_atendimentos_e_pessoas.csv")
dados_minio = response.read()
data_frame = pd.read_csv(io.BytesIO(dados_minio), encoding="latin1",  error_bad_lines=False)

dados = data_frame.iloc[:, 0].str.split(";", expand=True)
colunas = pd.Series(data_frame.columns.str.split(";"))

#print(type(["A", "B", "C", "D", "E", "F", "G"]))
#print(type(data_frame.columns.str.split(";")))

data_frame.columns.str.split(";")

#["A", "B", "C", "D", "E", "F", "G"]

#dados.rename(columns=pd.Series(["A", "B", "C", "D", "E", "F", "G"]))

dados.rename(columns={0: "DATA", 1: "NOME", 2: "SEXO", 3: "SIT", 4: "MUN", 5: "ATEND", 6: "STATUS"})


b'Skipping line 9106: expected 1 fields, saw 2\n'


,DATA,NOME,SEXO,SIT,MUN,ATEND,STATUS
0,01/01/2019,ISRAEL NASCIMENTO MATTOS,M,N,VITÓRIA,GP ORI,A
1,02/01/2019,ANDERSON RAILE DE OLIVEIRA ROSA,M,N,VILA VELHA,AVI COT,A
2,02/01/2019,DANIEL DOS SANTOS,M,S,VITÓRIA,AVI COT,A
3,02/01/2019,FRANCISCO CLERME NEVES,M,N,VITÓRIA,AVI COT,A
4,02/01/2019,GILBERTO SOUZA SOARES,M,N,SERRA,AVI COT,A
...,...,...,...,...,...,...,...
29763,28/10/2021,FERNANDA ACYPRESTE ALFREDO,F,N,SERRA,AMB,A
29764,28/10/2021,INAH SOUZA CORREIA,F,N,VITÓRIA,AMB,A
29765,28/10/2021,ADRIANO SANT ANA ULTRAMAR,M,N,CARIACICA,AMB,A
29766,29/10/2021,José Alzenir de Freitas,M,N,VILA VELHA,AMB,A


In [6]:
from minio import Minio
import pandas as pd
import io
import datetime

# criação do client minio
client = Minio("bi.s3.es.gov.br", access_key="BI@Prodest", secret_key="!BI@Prodest2021!")

# ler dados
response = client.get_object(bucket_name="sedh", object_name="OCID_INDICADORES_atendimentos_e_pessoas.csv")
dados_minio = response.read()
dados_saida = pd.read_csv(io.BytesIO(dados_minio), encoding="latin1", sep=";", error_bad_lines=False)
dados_saida

,DATA,NOME,SEXO,SITUAÇÃO DE RUA (S/N),MUNICIPIO,TIPO,ACO/FAM
0,01/01/2019,ISRAEL NASCIMENTO MATTOS,M,N,VITÓRIA,GP ORI,A
1,02/01/2019,ANDERSON RAILE DE OLIVEIRA ROSA,M,N,VILA VELHA,AVI COT,A
2,02/01/2019,DANIEL DOS SANTOS,M,S,VITÓRIA,AVI COT,A
3,02/01/2019,FRANCISCO CLERME NEVES,M,N,VITÓRIA,AVI COT,A
4,02/01/2019,GILBERTO SOUZA SOARES,M,N,SERRA,AVI COT,A
...,...,...,...,...,...,...,...
29764,28/10/2021,FERNANDA ACYPRESTE ALFREDO,F,N,SERRA,AMB,A
29765,28/10/2021,INAH SOUZA CORREIA,F,N,VITÓRIA,AMB,A
29766,28/10/2021,ADRIANO SANT ANA ULTRAMAR,M,N,CARIACICA,AMB,A
29767,29/10/2021,José Alzenir de Freitas,M,N,VILA VELHA,AMB,A


In [ ]:
#print(type(data_frame))

#data_frame = pd.DataFrame(data=data_frame, columns=data_frame.columns.str.split(";"))
data_frame.columns.str.split(";")
colunas = pd.Series(data_frame.columns.str.split(";"))
colunas

In [ ]:
data_frame.iloc[:, 0].str.split(";", expand=True)

In [ ]:
type(colunas)